In [1]:
import pandas as pd
import numpy as np
#import yfinance as yf
import datetime as datetime
#import ta as ta
import math
#import seaborn as sns
#import statsmodels.api as sm
#from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")
import pdb
from decimal import Decimal
from ibapi.client import *
from ibapi.wrapper import *
import threading

In [2]:
#from ..lib import utils, constants
import datetime
import time
import copy
#import signal
import sys
#import pickle

In [3]:


#python -m navis.research.getHistoricalData
#username: hzpsur161
#Flushit!234

port = 7497 ## 7496 is live, 7497 is paper trading
client_id = 150

duration = "1 M"
bar_size = "1 min"

#sp500_components = pd.read_csv("data/sp500_components.csv")

#pdb.set_trace()

In [4]:

class IBApi(EClient, EWrapper):

    def __init__(self, ib_client):
        EClient.__init__(self, self)
        self.ib_client = ib_client
        self.reqId = 500

    def getReqId(self):
        self.reqId += 1
        return self.reqId

    def historicalData(self, reqId:int, bar: BarData):
        print(bar)
        try:
            date_str = bar.date.split(" ")[0]
            time_str = bar.date.split(" ")[1].split(":")
            year_ = int(date_str[0:4])
            month_ = int(date_str[4:6])
            day_ = int(date_str[6:8])
            hour_ = int(time_str[0])
            minute_ = int(time_str[1])
            second_ = int(time_str[2])
            date_dt = datetime.datetime(year_, month_, day_, hour_, minute_, second_)
        except:
            date_str = bar.date.split(" ")[0]
            year_ = int(date_str[0:4])
            month_ = int(date_str[4:6])
            day_ = int(date_str[6:8])

            date_dt = datetime.date(year_, month_, day_)


        try:
            self.ib_client.df.loc[date_dt] = {"Datetime": date_dt, "Open": bar.open, "High": bar.high, "Low": bar.low, "Close": bar.close, "Volume": bar.volume}
        except:
            pass

    def historicalDataEnd(self, reqId: int, start: str, end: str):
        super().historicalDataEnd(reqId, start, end)
        print("HistoricalDataEnd. ReqId:", reqId, "from", start, "to", end)



In [5]:




class TV:

    def __init__(self):
        self.ib = IBApi(self)
        self.ib.connect("127.0.0.1", port, client_id)  ## 7496 is live, 7497 is paper trading
        ib_thread = threading.Thread(target=self.run_loop, daemon=True)
        ib_thread.start()

        self.cols = {'Datetime': int(), 'Open': float(), "High": float(), "Low": float(), "Close": float(), "Volume": float()}

    def run_loop(self):
        self.ib.run()

    def d1(self, strike, stock_price, vol, tte_in_days):
        return ( np.log(stock_price/strike) + 0.5*vol*vol*tte_in_days/252) / (vol * np.sqrt(tte_in_days/252))

    def delta(self, strike, stock_price, vol, call_put, tte_in_days):
        d_1 = self.d1(strike, stock_price, vol, tte_in_days)
        if call_put == 'call':
            delta = norm.cdf(d_1)
        else:
            delta = -1*norm.cdf(-1*d_1)
        return delta

    def getStrikeGivenDelta(self, target_delta, stock_price, call_put, tte_in_days):
        target_strike = np.nan
        vol = 0.2
        for strike in range(round(stock_price) - 10, round(stock_price)+10):
            delta_ = self.delta(strike, stock_price, vol, call_put ,tte_in_days)
            if target_delta-0.06 <= delta_ <= target_delta+0.06:
                target_strike = strike

        return target_strike

    def getStockContract(self,ticker):
        myC = Contract()
        myC.symbol = ticker
        myC.secType = "STK"
        myC.exchange = "SMART"
        myC.primaryExchange = "ISLAND"
        myC.currency = "USD"

        return myC

    def request_historical_data(self, reqId, ticker):

        self.df = pd.DataFrame(self.cols, [])

        _contract = self.getStockContract(ticker)
        time.sleep(1)
        self.ib.reqHistoricalData(reqId, _contract, "", duration, bar_size, "TRADES", 1, 1, True, [])
        time.sleep(1)


    def algo(self):

      #  tickers = sp500_components.Symbol.values
        #first, request historical data
        for ticker in ['SPY']:#, 'QQQ', 'IWM','AAPL','AMZN', 'GOOGL', 'META', 'MSFT','NVDA','TSLA']:
        #for ticker in np.append(tickers,['SPY', 'QQQ', 'IWM', 'GOOGL', 'AMZN', 'AAPL', 'MSFT', '']):
            reqId = self.ib.getReqId()
            self.request_historical_data(reqId, ticker)
            time.sleep(20)
            #self.df["Date"] = self.df.Datetime.apply(lambda x: x.date())
            print(self.df)
            #self.df.to_csv('C:/Users/rehan/navis/data/historical_intraday_data/historical_intraday_ul_%s_0.01.csv' % (ticker))

    def start(self):

        t1 = threading.Thread(target=self.algo, daemon=True)
        all_threads = [t1]
        for t in all_threads:
            t.start()
        for t in all_threads:
            t.join()

    def disconnect(self):
        self.ib.disconnect()

    def signal_handler(self, sig, _):
        print("CTRL C")
        self.disconnect()
        sys.exit(0)



In [ ]:


if __name__ == '__main__':

    tv = TV()
    try:
     # signal.signal(signal.SIGINT, tv.signal_handler)
        tv.start()
    except KeyboardInterrupt:
        pass
    finally:
        tv.disconnect()
